In [ ]:
# Refs:
# - https://github.com/jhayes14/adversarial-patch/tree/master
# - https://arxiv.org/abs/1712.09665
# - https://arxiv.org/pdf/2010.13070.pdf


In [ ]:
## Standard libraries
import os
import json
import math
import time
import numpy as np
import scipy.linalg

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Path to the folder where the datasets are/should be downloaded (e.g. MNIST)
DATASET_PATH = "/tmp/pycharm_project_70/data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "/tmp/pycharm_project_70/patches"

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

# Fetching the device that will be used throughout this notebook
device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

<ipython-input-2-f8420bf78b87>:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export
INFO:lightning_fabric.utilities.seed:Seed set to 42


Using device cuda:0


In [ ]:
import urllib.request
from urllib.error import HTTPError
import zipfile
# Github URL where the dataset is stored for this tutorial https://github.com/phlippe/saved_models/tree/main/tutorial10
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial10/"
# Files to download
pretrained_files = [(DATASET_PATH, "TinyImageNet.zip"), (CHECKPOINT_PATH, "patches.zip")]
# Create checkpoint path if it doesn't exist yet
os.makedirs(DATASET_PATH, exist_ok=True)
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# For each file, check whether it already exists. If not, try downloading it.
for dir_name, file_name in pretrained_files:
    file_path = os.path.join(dir_name, file_name)
    if not os.path.isfile(file_path):
        file_url = base_url + file_name
        print(f"Downloading {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
        except HTTPError as e:
            print("Something went wrong. Please try to download the file from the GDrive folder, or contact the author with the full output including the following error:\n", e)
        if file_name.endswith(".zip"):
            print("Unzipping file...")
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(file_path.rsplit("/",1)[0])

Unzipping file...
Unzipping file...


In [ ]:
# Load CNN architecture pretrained on ImageNet
os.environ["TORCH_HOME"] = CHECKPOINT_PATH
pretrained_model = torchvision.models.resnet34(pretrained=True)
pretrained_model = pretrained_model.to(device)

# No gradients needed for the network
pretrained_model.eval()
for p in pretrained_model.parameters():
    p.requires_grad = False

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /tmp/pycharm_project_70/patches/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 106MB/s]


In [ ]:
# Mean and Std from ImageNet
NORM_MEAN = np.array([0.485, 0.456, 0.406])
NORM_STD = np.array([0.229, 0.224, 0.225])
# No resizing and center crop necessary as images are already preprocessed.
plain_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN,
                         std=NORM_STD)
])

# Load dataset and create data loader
imagenet_path = os.path.join(DATASET_PATH, "TinyImageNet/")
assert os.path.isdir(imagenet_path), f"Could not find the ImageNet dataset at expected path \"{imagenet_path}\". "
dataset = torchvision.datasets.ImageFolder(root=imagenet_path, transform=plain_transforms)
data_loader = data.DataLoader(dataset, batch_size=32, shuffle=False, drop_last=False, num_workers=2)

# Load label names to interpret the label numbers 0 to 999
with open(os.path.join(imagenet_path, "label_list.json"), "r") as f:
    label_names = json.load(f)

def get_label_index(lab_str):
    assert lab_str in label_names, f"Label \"{lab_str}\" not found. Check the spelling of the class."
    return label_names.index(lab_str)

## 1. Regular patch code (from class)

In [ ]:
def place_patch(img, patch):
    for i in range(img.shape[0]):
        h_offset = np.random.randint(0,img.shape[2]-patch.shape[1]-1)
        w_offset = np.random.randint(0,img.shape[3]-patch.shape[2]-1)
        img[i,:,h_offset:h_offset+patch.shape[1],w_offset:w_offset+patch.shape[2]] = patch_forward(patch)
    return img

In [ ]:
TENSOR_MEANS, TENSOR_STD = torch.FloatTensor(NORM_MEAN)[:,None,None], torch.FloatTensor(NORM_STD)[:,None,None]
def patch_forward(patch):
    # Map patch values from [-infty,infty] to ImageNet min and max
    patch = (torch.tanh(patch) + 1 - 2 * TENSOR_MEANS) / (2 * TENSOR_STD)
    return patch

In [ ]:
def eval_patch(model, patch, val_loader, target_class):
    model.eval()
    tp, tp_5, counter = 0., 0., 0.
    with torch.no_grad():
        for img, img_labels in tqdm(val_loader, desc="Validating...", leave=False):
            # For stability, place the patch at 4 random locations per image, and average the performance
            for _ in range(4):
                patch_img = place_patch(img, patch)
                patch_img = patch_img.to(device)
                img_labels = img_labels.to(device)
                pred = model(patch_img)
                # In the accuracy calculation, we need to exclude the images that are of our target class
                # as we would not "fool" the model into predicting those
                tp += torch.logical_and(pred.argmax(dim=-1) == target_class, img_labels != target_class).sum()
                tp_5 += torch.logical_and((pred.topk(5, dim=-1)[1] == target_class).any(dim=-1), img_labels != target_class).sum()
                counter += (img_labels != target_class).sum()
    acc = tp/counter
    top5 = tp_5/counter
    return acc, top5

In [ ]:
def patch_attack(model, target_class, patch_size=64, num_epochs=5): #targeted
    # Leave a small set of images out to check generalization
    # In most of our experiments, the performance on the hold-out data points
    # was as good as on the training set. Overfitting was little possible due
    # to the small size of the patches.
    train_set, val_set = torch.utils.data.random_split(dataset, [4500, 500])
    train_loader = data.DataLoader(train_set, batch_size=32, shuffle=True, drop_last=True, num_workers=8)
    val_loader = data.DataLoader(val_set, batch_size=32, shuffle=False, drop_last=False, num_workers=4)

    # Create parameter and optimizer
    if not isinstance(patch_size, tuple):
        patch_size = (patch_size, patch_size)
    patch = nn.Parameter(torch.zeros(3, patch_size[0], patch_size[1]), requires_grad=True)
    optimizer = torch.optim.SGD([patch], lr=1e-1, momentum=0.8)
    loss_module = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(num_epochs):
        t = tqdm(train_loader, leave=False)
        for img, _ in t:
            img = place_patch(img, patch)
            img = img.to(device)
            pred = model(img)
            labels = torch.zeros(img.shape[0], device=pred.device, dtype=torch.long).fill_(target_class)
            loss = loss_module(pred, labels)
            optimizer.zero_grad()
            loss.mean().backward()
            optimizer.step()
            t.set_description(f"Epoch {epoch}, Loss: {loss.item():4.2f}")

    # Final validation
    acc, top5 = eval_patch(model, patch, val_loader, target_class)

    return patch.data, {"acc": acc.item(), "top5": top5.item()}

In [ ]:
# Load evaluation results of the pretrained patches
json_results_file = os.path.join(CHECKPOINT_PATH, "patch_results.json")
json_results = {}
if os.path.isfile(json_results_file):
    with open(json_results_file, "r") as f:
        json_results = json.load(f)



In [ ]:
# If you train new patches, you can save the results via calling this function
def save_results(patch_dict):
    result_dict = {cname: {str(psize): [t.item() if isinstance(t, torch.Tensor) else t
                                   for t in patch_dict[cname][psize]["results"]]
                           for psize in patch_dict[cname]}
                   for cname in patch_dict}
    with open(os.path.join(CHECKPOINT_PATH, "patch_results.json"), "w") as f:
        json.dump(result_dict, f, indent=4)

In [ ]:
def get_patches(class_names, patch_sizes):
    result_dict = dict()

    # Loop over all classes and patch sizes
    for name in class_names:
        result_dict[name] = dict()
        for patch_size in patch_sizes:
            c = label_names.index(name)
            file_name = os.path.join(CHECKPOINT_PATH, f"{name}_{patch_size}_patch.pt")
            # Load patch if pretrained file exists, otherwise start training
            if not os.path.isfile(file_name):
                patch, val_results = patch_attack(pretrained_model, target_class=c, patch_size=patch_size, num_epochs=5)
                print(f"Validation results for {name} and {patch_size}:", val_results)
                torch.save(patch, file_name)
            else:
                patch = torch.load(file_name)
            # Load evaluation results if exist, otherwise manually evaluate the patch
            if name in json_results and str(patch_size) in json_results[name]:
                results = json_results[name][str(patch_size)]
            else:
                results = eval_patch(pretrained_model, patch, data_loader, target_class=c)

            # Store results and the patches in a dict for better access
            result_dict[name][patch_size] = {
                "results": results,
                "patch": patch
            }

    return result_dict

Here we create patches in size (x, x/2) and (x/2, x) so that we can concatenate two patches later to create an x by x patch

In [ ]:
class_names = ['toaster', 'goldfish', 'school bus', 'lipstick', 'pineapple']
patch_sizes = [32, 48, 64, (32,16), (48,24), (64,32), (16,32), (24,48), (32,64)]

patch_dict = get_patches(class_names, patch_sizes)
# save_results(patch_dict) # Uncomment if you add new class names and want to save the new results

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

...

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

In [ ]:
save_results(patch_dict)

In [ ]:
def show_patches(patch_dict):
    num_classes = len(patch_dict)
    num_sizes = len(list(patch_dict.values())[0])
    fig, ax = plt.subplots(num_sizes, num_classes, figsize=(num_classes*2.2, num_sizes*2.2))
    for c_idx, cname in enumerate(patch_dict):
        for p_idx, psize in enumerate(patch_dict[cname]):
            patch = patch_dict[cname][psize]["patch"]
            patch = (torch.tanh(patch) + 1) / 2 # Parameter to pixel values
            patch = patch.cpu().permute(1, 2, 0).numpy()
            patch = np.clip(patch, a_min=0.0, a_max=1.0)
            ax[p_idx][c_idx].imshow(patch)
            ax[p_idx][c_idx].set_title(f"{cname}\n size {psize}")
            ax[p_idx][c_idx].axis('off')
    fig.subplots_adjust(hspace=0.3, wspace=0.3)
    plt.show()
show_patches(patch_dict)

In [ ]:
%%html
<!-- Some HTML code to increase font size in the following table -->
<style>
th {font-size: 120%;}
td {font-size: 120%;}
</style>

In [ ]:
import tabulate
from IPython.display import display, HTML

def show_table(top_1=True):
    i = 0 if top_1 else 1
    table = [[name] + [f"{(100.0 * patch_dict[name][psize]['results'][i]):4.2f}%" for psize in patch_sizes] for name in class_names]
    display(HTML(tabulate.tabulate(table, tablefmt='html', headers=["Class name"] + [f"Patch size {psize}" for psize in patch_sizes])))

In [ ]:
show_table(top_1=True)

In [ ]:
show_table(top_1=False)

In [ ]:
def perform_patch_attack(patch):
    patch_batch = exmp_batch.clone()
    patch_batch = place_patch(patch_batch, patch)
    with torch.no_grad():
        patch_preds = pretrained_model(patch_batch.to(device))
    for i in range(1,17,5):
        show_prediction(patch_batch[i], label_batch[i], patch_preds[i])

In [ ]:
def show_prediction(img, label, pred, K=5, adv_img=None, noise=None):

    if isinstance(img, torch.Tensor):
        # Tensor image to numpy
        img = img.cpu().permute(1, 2, 0).numpy()
        img = (img * NORM_STD[None,None]) + NORM_MEAN[None,None]
        img = np.clip(img, a_min=0.0, a_max=1.0)
        label = label.item()

    # Plot on the left the image with the true label as title.
    # On the right, have a horizontal bar plot with the top k predictions including probabilities
    if noise is None or adv_img is None:
        fig, ax = plt.subplots(1, 2, figsize=(10,2), gridspec_kw={'width_ratios': [1, 1]})
    else:
        fig, ax = plt.subplots(1, 5, figsize=(12,2), gridspec_kw={'width_ratios': [1, 1, 1, 1, 2]})

    ax[0].imshow(img)
    ax[0].set_title(label_names[label])
    ax[0].axis('off')

    if adv_img is not None and noise is not None:
        # Visualize adversarial images
        adv_img = adv_img.cpu().permute(1, 2, 0).numpy()
        adv_img = (adv_img * NORM_STD[None,None]) + NORM_MEAN[None,None]
        adv_img = np.clip(adv_img, a_min=0.0, a_max=1.0)
        ax[1].imshow(adv_img)
        ax[1].set_title('Adversarial')
        ax[1].axis('off')
        # Visualize noise
        noise = noise.cpu().permute(1, 2, 0).numpy()
        noise = noise * 0.5 + 0.5 # Scale between 0 to 1
        ax[2].imshow(noise)
        ax[2].set_title('Noise')
        ax[2].axis('off')
        # buffer
        ax[3].axis('off')

    if abs(pred.sum().item() - 1.0) > 1e-4:
        pred = torch.softmax(pred, dim=-1)
    topk_vals, topk_idx = pred.topk(K, dim=-1)
    topk_vals, topk_idx = topk_vals.cpu().numpy(), topk_idx.cpu().numpy()
    ax[-1].barh(np.arange(K), topk_vals*100.0, align='center', color=["C0" if topk_idx[i]!=label else "C2" for i in range(K)])
    ax[-1].set_yticks(np.arange(K))
    ax[-1].set_yticklabels([label_names[c] for c in topk_idx])
    ax[-1].invert_yaxis()
    ax[-1].set_xlabel('Confidence')
    ax[-1].set_title('Predictions')

    plt.show()
    plt.close()

In [ ]:
exmp_batch, label_batch = next(iter(data_loader))

In [ ]:
perform_patch_attack(patch_dict['goldfish'][(48,24)]['patch'])

# 2 Concatenate 2 regular patches, one rotated

The idea here is to take a class1 x by x/2 patch and concatenate it to a 90 degrees rotated (anti-clockwise) class2 x/2 by x patch. We hope that the resulting patch will predict class1 and when we will rotate it by 90 degrees clockwise it will predict class 2.

In [ ]:
def concat_patches_with_rot(patch1, patch2):
    return torch.cat([patch1, patch2.rot90(k=-1, dims=[-2,-1])], dim=-1)

Create the concatenated patches

In [ ]:
con_patches = {}
for i, class1 in enumerate(class_names):
    for class2 in class_names[i+1:]:
        con_patch_class = class1 + ", " + class2
        con_patches[con_patch_class] = {}
        for size1 in [(32,16), (48,24), (64,32)]:
            size2 = size1[::-1]
            patch1 = patch_dict[class1][size1]['patch']
            patch2 = patch_dict[class2][size2]['patch']
            con_patch = concat_patches_with_rot(patch1, patch2)
            con_patch_size = size1[0]
            con_patches[con_patch_class][con_patch_size] = {}
            con_patches[con_patch_class][con_patch_size]['patch'] = con_patch

In [ ]:
show_patches(con_patches)

An helper function to save the evaluations of the concatenated patches

In [ ]:
def save_results_con(con_patches):
    result_dict = {cname: {str(psize): [[t1.item() if isinstance(t1, torch.Tensor) else t1,
                                         t2.item() if isinstance(t2, torch.Tensor) else t2,]
                  for t1,t2 in con_patches[cname][psize]["results"]]
                  for psize in con_patches[cname] if "results" in con_patches[cname][psize]}
                  for cname in con_patches}
    with open(os.path.join(CHECKPOINT_PATH, "con_patch_results.json"), "w") as f:
        json.dump(result_dict, f, indent=4)

Evaluate the patches and save the results

In [ ]:
# Load evaluation results of the pretrained patches
json_con_results_file = os.path.join(CHECKPOINT_PATH, "con_patch_results.json")
json_con_results = {}
if os.path.isfile(json_con_results_file):
    with open(json_con_results_file, "r") as f:
        json_con_results = json.load(f)

for classes in tqdm(con_patches):
    class1, class2 = classes.split(", ")
    c1 = label_names.index(class1)
    c2 = label_names.index(class2)
    for patch_size in con_patches[classes]:
        patch1 = con_patches[classes][patch_size]['patch']
        patch2 = patch1.rot90(k=1, dims=[-2,-1])
        if classes in json_con_results and str(patch_size) in json_con_results[classes]:
            res1, res2 = json_con_results[classes][str(patch_size)]
        else:
            res1 = eval_patch(pretrained_model, patch1, data_loader, target_class=c1)
            res2 = eval_patch(pretrained_model, patch2, data_loader, target_class=c2)
        con_patches[classes][patch_size]['results'] = (res1, res2)
save_results_con(con_patches)

In [ ]:
def res_string(name, psize, j, i):
    return f"{(100.0 * con_patches[name][psize]['results'][j][i]):2.0f}%"

def show_table_con(top_1=True):
    i = 0 if top_1 else 1
    table = [[name] + [res_string(name, psize, 0, i) + ", " + res_string(name, psize, 1, i)
                        for psize in con_patches[name]] for name in con_patches]
    display(HTML(tabulate.tabulate(table, tablefmt='html', headers=["Classes"] +
                [f"Patch size {psize}" for psize in patch_sizes])))

Lets see the results:

The left percentage is the top 1 accurracy of the first class, when the patch is not rotated.

The right percentage is the top 1 accurracy of the second class, when the patch is rotated.

We can see this works well for patch size 64, with most accuracies around 70%-90%.

For 32 and 48, the accurracies are substantially lower.

In [ ]:
show_table_con(top_1=True)

Same table but with top 5 accuracy:

In [ ]:
show_table_con(top_1=False)

The following function presents a comparison between the different types of patches.

For a class c and size x:

"full" refers to the accuracy of the x by x patch dedicated for class c only.

"half" refers to the average accuracy of the x by x/2 and the x/2 by x patches dedicated for class c only.

"concat" refers to the average accuracy of the x by x patches dedicated for class c and to another class (when rotated).


In [ ]:
def compare_patch_types(top_1=True):
    i = 0 if top_1 else 1
    ptypes = ['full', 'half', 'concat']
    res = {name:{psize:{k:0 for k in ptypes} for psize in [32,48,64]} for name in class_names}
    for classes in con_patches:
        class1, class2 = classes.split(", ")
        for psize in con_patches[classes]:
            res[class1][psize]['concat'] += con_patches[classes][psize]['results'][0][i] / (len(class_names) - 1)
            res[class2][psize]['concat'] += con_patches[classes][psize]['results'][1][i] / (len(class_names) - 1)

    for class_name in patch_dict:
        for psize in patch_dict[class_name]:
            if type(psize) == tuple:
                big_dim = max(psize)
                res[class_name][big_dim]['half'] += patch_dict[class_name][psize]['results'][i] / 2
            else:
                res[class_name][psize]['full'] = patch_dict[class_name][psize]['results'][i]
    table = [[name] + ["(" + str(psize)+")"] + [f"{(100.0 * res[name][psize][ptype]):4.2f}%" for ptype in ptypes]
                        for name in class_names for psize in [32,48,64]]
    display(HTML(tabulate.tabulate(table, tablefmt='html', headers=["Class name"] +
               ["Patch size"] + [f"Patch type: {ptype}" for ptype in ptypes])))

Lets see the results:

We can see that the "concat" accuracy is always lower than the "half" accuracy. For around half the cases the difference is not so big (around 10% or lower)

In [ ]:
compare_patch_types(top_1=True)

In [ ]:
compare_patch_types(top_1=False)

In [ ]:
perform_patch_attack(con_patches['lipstick, pineapple'][64]['patch'])

In [ ]:
perform_patch_attack(con_patches['lipstick, pineapple'][64]['patch'].rot90(dims=[-2,-1]))

# 3 Create a patch that shift to class A and when roateted shift to class B

In [ ]:
def place_patch(img, patch):
    for i in range(img.shape[0]):
        h_offset = np.random.randint(0,img.shape[2]-patch.shape[1]-1)
        w_offset = np.random.randint(0,img.shape[3]-patch.shape[2]-1)
        img[i,:,h_offset:h_offset+patch.shape[1],w_offset:w_offset+patch.shape[2]] = patch_forward(patch)
        img_rot = img.clone()
        img_rot[i,:,h_offset:h_offset+patch.shape[1],w_offset:w_offset+patch.shape[2]] = patch_forward(patch.rot90(k=1, dims=[-2,-1]))

    return img, img_rot

In [ ]:
TENSOR_MEANS, TENSOR_STD = torch.FloatTensor(NORM_MEAN)[:,None,None], torch.FloatTensor(NORM_STD)[:,None,None]
def patch_forward(patch):
    # Map patch values from [-infty,infty] to ImageNet min and max
    patch = (torch.tanh(patch) + 1 - 2 * TENSOR_MEANS) / (2 * TENSOR_STD)
    return patch

In [ ]:
def eval_patch(model, patch, val_loader, target_class):
    model.eval()
    tp, tp_5, counter = 0., 0., 0.
    with torch.no_grad():
        for img, img_labels in tqdm(val_loader, desc="Validating...", leave=False):
            # For stability, place the patch at 4 random locations per image, and average the performance
            for _ in range(4):
                patch_img, _ = place_patch(img, patch)
                patch_img = patch_img.to(device)
                img_labels = img_labels.to(device)
                pred = model(patch_img)
                # In the accuracy calculation, we need to exclude the images that are of our target class
                # as we would not "fool" the model into predicting those
                tp += torch.logical_and(pred.argmax(dim=-1) == target_class, img_labels != target_class).sum()
                tp_5 += torch.logical_and((pred.topk(5, dim=-1)[1] == target_class).any(dim=-1), img_labels != target_class).sum()
                counter += (img_labels != target_class).sum()
    acc = tp/counter
    top5 = tp_5/counter
    return acc, top5

In [ ]:
def patch_attack(model, target_class, target_class_rot, patch_size=64, num_epochs=5):
    # Leave a small set of images out to check generalization
    # In most of our experiments, the performance on the hold-out data points
    # was as good as on the training set. Overfitting was little possible due
    # to the small size of the patches.
    train_set, val_set = torch.utils.data.random_split(dataset, [4500, 500])
    train_loader = data.DataLoader(train_set, batch_size=32, shuffle=True, drop_last=True, num_workers=4)
    val_loader = data.DataLoader(val_set, batch_size=32, shuffle=False, drop_last=False, num_workers=4)

    # Create parameter and optimizer
    if not isinstance(patch_size, tuple):
        patch_size = (patch_size, patch_size)
    patch = nn.Parameter(torch.zeros(3, patch_size[0], patch_size[1]), requires_grad=True)
    optimizer = torch.optim.SGD([patch], lr=1e-1, momentum=0.8)
    loss_module = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(num_epochs):
        t = tqdm(train_loader, leave=False)
        for img, _ in t:
            img, img_rot = place_patch(img, patch)
            img, img_rot = img.to(device), img_rot.to(device)


            pred, pred_rot = model(img), model(img_rot)
            labels = torch.zeros(img.shape[0], device=pred.device, dtype=torch.long).fill_(target_class)
            labels_rot = torch.zeros(img_rot.shape[0], device=pred.device, dtype=torch.long).fill_(target_class_rot)

            # Combined loss
            loss = loss_module(pred, labels) + loss_module(pred_rot, labels_rot)
            optimizer.zero_grad()
            loss.mean().backward()
            optimizer.step()
            t.set_description(f"Epoch {epoch}, Loss: {loss.item():4.2f}")

    # Final validation
    acc, top5 = eval_patch(model, patch, val_loader, target_class)
    acc_rot, top5_rot = eval_patch(model, patch.rot90(k=1, dims=[-2,-1]), val_loader, target_class_rot)

    return patch.data, {"acc": acc.item(), "top5": top5.item(), "acc_rot": acc_rot.item(), "top5_rot": top5_rot.item()}

In [ ]:
# Load evaluation results of the pretrained patches
json_results_file = os.path.join(CHECKPOINT_PATH, "patch_results.json")
json_results = {}
if os.path.isfile(json_results_file):
    with open(json_results_file, "r") as f:
        json_results = json.load(f)

# If you train new patches, you can save the results via calling this function
def save_results(patch_dict):
    result_dict = {cname: {psize: [t.item() if isinstance(t, torch.Tensor) else t
                                   for t in patch_dict[cname][psize]["results"]]
                           for psize in patch_dict[cname]}
                   for cname in patch_dict}
    with open(os.path.join(CHECKPOINT_PATH, "patch_results.json"), "w") as f:
        json.dump(result_dict, f, indent=4)

In [ ]:
def get_patches(class_names, class_names_rot, patch_sizes):
    result_dict = dict()

    # Loop over all classes and patch sizes
    for name in class_names:
        for name_rot in class_names_rot:
            name_ = name + name_rot
            result_dict[name_] = dict()
            for patch_size in patch_sizes:
                c = label_names.index(name)
                c_rot = label_names.index(name_rot)
                file_name = os.path.join(CHECKPOINT_PATH, f"{name_}_{patch_size}_patch.pt")
                # Load patch if pretrained file exists, otherwise start training
                if not os.path.isfile(file_name):
                    patch, val_results = patch_attack(pretrained_model, target_class=c, target_class_rot=c_rot, patch_size=patch_size,
                                                      num_epochs=5)
                    print(f"Validation results for {name_} and {patch_size}:", val_results)
                    torch.save(patch, file_name)
                else:
                    patch = torch.load(file_name)
                # Load evaluation results if exist, otherwise manually evaluate the patch
                if name_ in json_results:
                    results = json_results[name_][str(patch_size)]
                else:
                    results = eval_patch(pretrained_model, patch, data_loader, target_class=c) # TODO: Update

                # Store results and the patches in a dict for better access
                result_dict[name_][patch_size] = {
                    "results": results,
                    "patch": patch
                }

    return result_dict

In [ ]:
class_names = ['goldfish', 'toaster', 'school bus'] # ['toaster', 'goldfish', 'school bus', 'lipstick', 'pineapple']
class_names_rot = ['lipstick', 'pineapple']
patch_sizes = [32, 48, 64]

patch_dict = get_patches(class_names, class_names_rot, patch_sizes)
# save_results(patch_dict) # Uncomment if you add new class names and want to save the new results]

In [ ]:
def show_patches():
    fig, ax = plt.subplots(len(patch_sizes), len(class_names) + len(class_names_rot),
                           figsize=((len(class_names) + len(class_names_rot))*2.2, len(patch_sizes)*2.2))
    for c_idx, cname in enumerate(class_names):
        for cname_rot in class_names_rot:
            for p_idx, psize in enumerate(patch_sizes):
                patch = patch_dict[cname + cname_rot][psize]["patch"]
                patch = (torch.tanh(patch) + 1) / 2 # Parameter to pixel values
                patch = patch.cpu().permute(1, 2, 0).numpy()
                patch = np.clip(patch, a_min=0.0, a_max=1.0)
                ax[p_idx][c_idx].imshow(patch)
                ax[p_idx][c_idx].set_title(f"{cname}, size {psize}")
                ax[p_idx][c_idx].axis('off')
    fig.subplots_adjust(hspace=0.3, wspace=0.3)
    plt.show()
show_patches()

In [ ]:
%%html
<!-- Some HTML code to increase font size in the following table -->
<style>
th {font-size: 120%;}
td {font-size: 120%;}
</style>

In [ ]:
import tabulate
from IPython.display import display, HTML

def show_table(top_1=True):
    i = 0 if top_1 else 1
    table = [[name] + [f"{(100.0 * patch_dict[name][psize]['results'][i]):4.2f}%" for psize in patch_sizes]
             for name in class_names]
    display(HTML(tabulate.tabulate(table, tablefmt='html', headers=["Class name"] + [f"Patch size {psize}x{psize}" for psize in patch_sizes])))

In [ ]:
show_table(top_1=True)

In [ ]:
show_table(top_1=False)

In [ ]:
def perform_patch_attack(patch):
    patch_batch = exmp_batch.clone()
    patch_batch = place_patch(patch_batch, patch)
    with torch.no_grad():
        patch_preds = pretrained_model(patch_batch.to(device))
    for i in range(1,17,5):
        show_prediction(patch_batch[i], label_batch[i], patch_preds[i])

In [ ]:
perform_patch_attack(patch_dict['goldfish'][32]['patch'])

In [ ]:
perform_patch_attack(patch_dict['school bus'][64]['patch'])

In [ ]:
import torch
import matplotlib.pyplot as plt

# Function to visualize tensors
def visualize_tensor(tensor, title):
    plt.imshow(tensor.squeeze().cpu().numpy(), cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Create random tensors as an example
patch1 = torch.randint(0, 10, (1, 1, 4, 4))  # Assuming a 4x4 patch
patch2 = torch.randint(0, 10, (1, 1, 4, 4))  # Assuming a 4x4 patch

# Visualize original tensors
visualize_tensor(patch1, "Patch 1")
visualize_tensor(patch2, "Patch 2")

# Concatenate patch1 with a rotated version of patch2
concatenated_tensor = torch.cat([patch1, patch2.rot90(k=-1, dims=[-2,-1])], dim=-1)

# Visualize concatenated tensor
visualize_tensor(concatenated_tensor, "Concatenated Tensor")
